# Pump it Up: Tanzania Water Wells

**Authors:** Kevin McDonough, Ryan Reilly

![title](images/HappyKid_well.jpeg)

## Overview

This project analyzes data for over 74,000 water wells that have been installed in Tanzania over the years. The goal of this analysis is to determine what features of a water well provide a good prediction if the well is functional, not functional, or functional but needs repair. This will be done through exploratory data analysis and iterative predictive modeling using classification models. 

## Business Problem

The Tanzania Ministry of Water have hired us to predict the operating condition for wells in their country. They will use our analysis to send teams of people out to fix the waterpoints that are currently not functional or need repair. Based on our analysis, we are going to provide reccomendations based on the following.

- Which wells you should start fixing first based on location
- Which funders and installers to focus on when building new wells
- Which type of wells should be used to replace non-functional wells
- Who should manage the wells

## Data Understanding

Each row in this dataset represents a unique water well in Tanzania and surrounding information about the well. There are a number of columns related to geo location of the well. There is also a good mix of continuous and categorical variables in the dataset. Each feature and its description is listed below.

The below cell is used to left align the table

In [10]:
%%html
<style>
table {float:left}
</style>

| Feature | Description|
|:------- | :-------|
|amount_tsh| Total static head (amount water available to waterpoint)|
|date_recorded| The date the row was entered|
|funder| Who funded the well|
|gps_height| Altitude of the well|
|installer| Organization that installed the well|
|longitude| GPS coordinate|
|latitude| GPS coordinate|
|wpt_name| Name of the waterpoint if there is one|
|num_private| No info for this column|
|basin| Geographic water basin|
|subvillage| Geographic location|
|region| Geographic location|
|region_code| Geographic location (coded)|
|district_code| Geographic location (coded)|
|lga| Geographic location|
|ward| Geographic location|
|population| Population around the well|
|public_meeting| True/False|
|recorded_by| Group entering this row of data|
|scheme_management| Who operates the waterpoint|
|scheme_name| Who operates the waterpoint|
|permit| If the waterpoint is permitted|
|construction_year| Year the waterpoint was constructed|
|extraction_type| The kind of extraction the waterpoint uses|
|extraction_type_group| The kind of extraction the waterpoint uses|
|extraction_type_class| The kind of extraction the waterpoint uses|
|management| How the waterpoint is managed|
|management_group| How the waterpoint is managed|
|payment| What the water costs|
|payment_type| What the water costs|
|water_quality| The quality of the water|
|quality_group| The quality of the water|
|quantity| The quantity of water|
|quantity_group| The quantity of water|
|source| The source of the water|
|source_type| The source of the water|
|source_class| The source of the water|
|waterpoint_type| The kind of waterpoint|
|waterpoint_type_group| The kind of waterpoint|

#### Importing packages and loading the datasets

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.dummy import DummyClassifier

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline

import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import json

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import time

import warnings
warnings.filterwarnings('ignore')

In [13]:
#Import the data
X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')

#### Get an understanding of the data

In [18]:
#Take a look at the datatypes
X_train.info()
print('---------------------------------------------------')
print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

From the info above, it looks like we have 59,400 rows in our training data representing water wells with 39 columns. We will have to decide how we want to treat the columns with nulls which are funder, installer, subvillage, public_meeting, scheme_management, sheme_name, and permit. Most of our data types are objects, so we will need to encode a lot of these variables to for our classification models. There also looks to be a lot of columns that may have the same type of information (ie, waterpoint_type and waterpoint_type_group) so we will explore these value_counts. 

In [55]:
#Take a look at the outcome variable and its value counts
y_train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

There are three classes. There looks to be a good balance of functional and non functional values, but there is a low sample of functional needs repair. Since this is pretty imbalanced, we may need to implement a resampling techinique to synthitically create more values in this third class. We may use SMOTE when we get to the modeling stage. 

## Data Preparation

### Handle Missing Values

In [54]:
#Below are the columns with missing values

missing = pd.DataFrame(X_train.isna().sum(), columns = ['Nulls'])
    
missing.sort_values(by=['Nulls'])[-7:]

,Nulls
subvillage,371
permit,3056
public_meeting,3334
funder,3635
installer,3655
scheme_management,3877
scheme_name,28166


#### funder

#### installer

#### subvillage

#### public_meeting

#### scheme_management

#### scheme_name

#### permit

### Handle Outliers

In [56]:
X_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


#### construction_year

#### population

#### gps_height

#### latitude

#### longitude

### Check for row duplicates

In [ ]:
X_train[X_train.duplicated()]

In [ ]:
X_test[X_test.duplicated()]

### Remove columns

#### Check value counts of categorical variables to see if information is the same between columnsf

In [ ]:
X_train.drop(['amount_tsh', 'num_private', 'region', 'quantity_group', 
              'source_type', 'payment_type', 'waterpoint_type_group', 
              'extraction_type_group', 'extraction_type_class','recorded_by'], 
               axis = 'columns', inplace=True)

#### Reasons for removal: 

id is not a useful predictor

amount_tsh is ~70% zeros

num_private is ~99% zeros

region is the same as region_code but we will keep region code becasue it is already numeric and ready for our models.

quantity_group has the same values and value counts as quantity

source_type has roughly the same categories as source, but source has the categories broken out better (river and lake are seperated in source)

payment_type has the same values and value counts as payment

waterpoint_type_group has the same values and value counts as waterpoint_type

extraction_type_group has roughly the same categories as extraction_type, but extraction_type has the categories broken out better

extraction_type_class has even even less catagories than extraction_type_group

recorded_by has just one value, "GeoData Consultants Ltd", so it would not be useful in EDA or prediction.

# Feature Engineering

#### Date Columns

In [57]:
#Convert the date_recorded to datetime column
X_train['date_recorded'] = pd.to_datetime(X_train['date_recorded'])

In [58]:
#Create the month and month number column from the date column
X_train['month_recorded'] = X_train['date_recorded'].dt.month_name()
X_train['month_num_recorded'] = X_train['date_recorded'].dt.month

In [59]:
#Create the year recorded from date column
X_train['year_recorded'] = X_train['date_recorded'].dt.year

# Exploratory Data Analysis

# Train test split

# Modeling

# Conclusions

# Next Steps